In [ ]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


Загружаем наши данные

In [ ]:
df = pd.read_csv('../input/thoraric-surgery/ThoraricSurgery.csv', index_col = 'id')

In [ ]:
df.head(2)


Data Set Information:

The data was collected retrospectively at Wroclaw Thoracic Surgery Centre for patients who underwent major lung resections for primary lung cancer in the years 2007â€“2011. The Centre is associated with the Department of Thoracic Surgery of the Medical University of Wroclaw and Lower-Silesian Centre for Pulmonary Diseases, Poland, while the research database constitutes a part of the National Lung Cancer Registry, administered by the Institute of Tuberculosis and Pulmonary Diseases in Warsaw, Poland.


Attribute Information:

1. DGN: Diagnosis - specific combination of ICD-10 codes for primary and secondary as well multiple tumours if any (DGN3,DGN2,DGN4,DGN6,DGN5,DGN8,DGN1)
2. PRE4: Forced vital capacity - FVC (numeric)
3. PRE5: Volume that has been exhaled at the end of the first second of forced expiration - FEV1 (numeric)
4. PRE6: Performance status - Zubrod scale (PRZ2,PRZ1,PRZ0)
5. PRE7: Pain before surgery (T,F)
6. PRE8: Haemoptysis before surgery (T,F)
7. PRE9: Dyspnoea before surgery (T,F)
8. PRE10: Cough before surgery (T,F)
9. PRE11: Weakness before surgery (T,F)
10. PRE14: T in clinical TNM - size of the original tumour, from OC11 (smallest) to OC14 (largest) (OC11,OC14,OC12,OC13)
11. PRE17: Type 2 DM - diabetes mellitus (T,F)
12. PRE19: MI up to 6 months (T,F)
13. PRE25: PAD - peripheral arterial diseases (T,F)
14. PRE30: Smoking (T,F)
15. PRE32: Asthma (T,F)
16. AGE: Age at surgery (numeric)
17. Risk1Y: 1 year survival period - (T)rue value if died (T,F)

Произведем следующие преобразования для повышения читабельности датасета

In [ ]:
#T/F = 1/0
df[['PRE7', 'PRE8', 'PRE9', 'PRE10', 'PRE11', 'PRE17', 'PRE19', 'PRE25', 'PRE30', 'PRE32', 'Risk1Yr']] = \
(df[['PRE7', 'PRE8', 'PRE9', 'PRE10', 'PRE11', 'PRE17', 'PRE19', 'PRE25', 'PRE30', 'PRE32', 'Risk1Yr']] == 'T').astype(int)

In [ ]:
#в ячейках содержащих числовые и строковые данные - оставим только цифры
df['DGN']   = df['DGN'].str[-1:].astype(int)
df['PRE6']  = df['PRE6'].str[-1:].astype(int)
df['PRE14'] = df['PRE14'].str[-1:].astype(int)

In [ ]:
#переименовываем
columns = ['Diagnosis','Forced_Capacity','Forced_Expiration','Zubrod_scale','Pain',' Haemoptysis','Dyspnoea',
       'Cough','Weakness','Size_of_tumor','diabetes','MI_6months','PAD','Smoker','Asthmatic','Age','Risk_1y']
df.columns = columns

In [ ]:
df.head(2)

In [ ]:
from pandas_profiling import ProfileReport
report = ProfileReport(df, minimal = False, progress_bar=True)

In [ ]:
report.to_notebook_iframe()

Данных c диагнозами очень очень мало
Объединим диагнозы 1,5,6,7,8 в один, т.к. по ним мало данных - и они существенного правильного прогноза не дадут (низкая точность)

In [ ]:
data = df.copy()
data['Diagnosis'] = np.where(df['Diagnosis'].isin([1,5,6,7,8]), 0, df['Diagnosis'])
data.head(3)

In [ ]:
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.model_selection import learning_curve, GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Ridge, LinearRegression, LogisticRegression
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor

import warnings
warnings.filterwarnings("ignore")

In [ ]:
X = data.drop(columns='Diagnosis')
y = data.Diagnosis

пусть базовое предсказание будет без аугментации - обычная логистическая регрессия

In [ ]:
clf = LogisticRegression(class_weight = 'balanced')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print(metrics.classification_report(y_test, predictions))

т.к по диагнозу 3 было больше всего наблюдений  там получилось более менее точные предсказания

Проведем аугментацию данных

In [ ]:
from imblearn.over_sampling import SMOTE, ADASYN

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

augm = ADASYN()
X_train_augm, y_train_augm = augm.fit_resample(np.array(X_train), np.array(y_train))

In [ ]:
#исходные данные
pd.Series(y_train).value_counts()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
plt.figure(figsize=(6, 4))
sns.countplot(y_train)
plt.xlabel('Class')
plt.ylabel('Count')
plt.tight_layout()

In [ ]:
#данные после аугментации
pd.Series(y_train_augm).value_counts()

In [ ]:
plt.figure(figsize=(6, 4))
sns.countplot(y_train_augm)
plt.xlabel('Class')
plt.ylabel('Count')
plt.tight_layout()

In [ ]:
import collections
collections.Counter(y_train_augm)

Произведем прогноз с использованием логистической регрессии аугментированных данных

In [ ]:
clf = LogisticRegression()
clf.fit(X_train_augm, y_train_augm)
predictions = clf.predict(X_test)
print(metrics.classification_report(y_test, predictions))

In [ ]:
predictions_ground_truth_df = pd.DataFrame(list(zip(predictions, y_train_augm)))
predictions_ground_truth_df.columns = ['Prediction', 'Ground_truth']
predictions_ground_truth_df.head()


In [ ]:
plt.figure(figsize=(6, 4))
plt.scatter(predictions_ground_truth_df.Prediction, predictions_ground_truth_df.Ground_truth, c = '#ad09a3')
plt.xlabel('Predicted')
plt.ylabel('Ground truth')
plt.plot([0, 5], [0, 5], color="red")
plt.tight_layout()


Построим другие модели по аугментированным данным для получения улучшенного результата

In [ ]:
#KNN
knn = KNeighborsClassifier()
knn.fit(X_train_augm, y_train_augm)
predictions = knn.predict(X_test)
print(metrics.classification_report(y_test, predictions))

In [ ]:
#random forest
rfc = RandomForestClassifier(random_state=666)
rfc.fit(X_train_augm, y_train_augm)
predictions = rfc.predict(X_test)
print(metrics.classification_report(y_test, predictions))

# Метод опорных векторов (SVM)
Мы не можем заранее знать, какой тип ядра предсказывает наилучшие результаты - поэтому попробуем несколько различных типов ядер.

In [ ]:
# linear svm:
svm = SVC(kernel='linear', probability=True)
svm.fit(X_train_augm, y_train_augm)
predictions = svm.predict(X_test)
print(metrics.classification_report(y_test, predictions))

In [ ]:
# linear rbf:
svm = SVC(kernel='rbf', probability=True, gamma=10)
svm.fit(X_train_augm, y_train_augm)
predictions = svm.predict(X_test)
print(metrics.classification_report(y_test, predictions))

In [ ]:
#  poly svm:
svm = SVC(kernel='poly', probability=True)
svm.fit(X_train_augm, y_train_augm)
predictions = svm.predict(X_test)
print(metrics.classification_report(y_test, predictions))

Лучший показатель при применении алгоритма SVM показало простейшее линейное ядро. 
Данный алгоритм хорошие резульататы показывает на небольших данных. В нашей модели он показал лучший результат т.к. данные наши линейно сепарабельные
Гаусовское и полиномиальное ядра хорошо себя показывают на больших данных и разнообразных признаках.

# Оптимизация гиперпараметров. RandomizedSearchCV ч.1
на базе random forest

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
max_features = ['log2', 'sqrt']
max_depth = [int(x) for x in np.linspace(start = 1, stop = 15, num = 15)]
min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 50, num = 10)]
min_samples_leaf = [int(x) for x in np.linspace(start = 2, stop = 50, num = 10)]
bootstrap = [True, False]
param_dist = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rs = RandomizedSearchCV(rfc, 
                        param_dist, 
                        n_iter = 100, 
                        cv = 3, 
                        verbose = 1, 
                        n_jobs=-1, 
                        random_state=0)
rs.fit(X_train_augm, y_train_augm)
rs.best_params_


При значениях параметров n_iter = 100 и cv = 3, мы создали 300 RF-моделей, случайно выбирая комбинации представленных выше гиперпараметров. Мы можем обратиться к атрибуту best_params_ для получения сведений о наборе параметров, позволяющем создать самую лучшую модель. Но на данной стадии это может не дать нам наиболее интересных данных о диапазонах параметров, которые стоит изучить на следующем раунде оптимизации. Для того чтобы выяснить то, в каком диапазоне значений стоит продолжать поиск, мы легко можем получить датафрейм, содержащий результаты работы алгоритма RandomizedSearchCV.

In [ ]:
rs_df = pd.DataFrame(rs.cv_results_).sort_values('rank_test_score').reset_index(drop=True)
rs_df = rs_df.drop([
            'mean_fit_time', 
            'std_fit_time', 
            'mean_score_time',
            'std_score_time', 
            'params', 
            'split0_test_score', 
            'split1_test_score', 
            'split2_test_score', 
            'std_test_score'],
            axis=1)
rs_df.head(5)

In [ ]:
fig, axs = plt.subplots(ncols=3, nrows=2)
sns.set(style="whitegrid", color_codes=True, font_scale = 2)
fig.set_size_inches(30,25)
sns.barplot(x='param_n_estimators', y='mean_test_score', data=rs_df, ax=axs[0,0], color='lightgrey')
sns.barplot(x='param_min_samples_split', y='mean_test_score', data=rs_df, ax=axs[0,1], color='coral')
sns.barplot(x='param_min_samples_leaf', y='mean_test_score', data=rs_df, ax=axs[0,2], color='lightgreen')
sns.barplot(x='param_max_features', y='mean_test_score', data=rs_df, ax=axs[1,0], color='wheat')
sns.barplot(x='param_max_depth', y='mean_test_score', data=rs_df, ax=axs[1,1], color='lightpink')
sns.barplot(x='param_bootstrap',y='mean_test_score', data=rs_df, ax=axs[1,2], color='skyblue')
plt.show()

используя то, что мы выяснили с помощью RandomizedSearchCV, исследуем значения гиперпараметров, которые лучше всего себя показали

In [ ]:
from sklearn.model_selection import GridSearchCV
n_estimators = [500,700]
max_features = ['sqrt']
max_depth = [7, 11, 15]
min_samples_split = [2,12,23,44]
min_samples_leaf = [2,7, 18]
bootstrap = [False]
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
gs = GridSearchCV(rfc, param_grid, cv = 3, verbose = 1, n_jobs=-1)
gs.fit(X_train_augm, y_train_augm)
predictions = gs.predict(X_test)
print(metrics.classification_report(y_test, predictions))

**Вывод:**
При решении нашей задачи классификации на маленьких данных мы смогли сделать хорошие прогнозы по 3 и 4 диагнозу с помощью применения линейных моделей, т.к. категориальные признаки по данных диагнозам находятся в линейной зависимости, имеют низкий разброс и исходных данных для "обучения с учителем" было больше всего. Все остальные диагнозы получили практически случайное предсказание.